<span style="font-family: Arial; font-weight:bold;font-size:2em;color:#0ab6fa">Presolve Reductions for LPs with Gurobi

**Phillipe Vilaça Gomes** <br>
**Institute for Research in Technology IIT, Comillas Pontifical University** <br>
**Madrid, Spain, 2024.** <br>
- Phillipe Vilaça, phillipe.v@comillas.edu

**Note**: <br>
This educational material on "Presolve Reductions for LPs with Gurobi" is derived from the "Interpretable Optimization" project. This project was conducted by: <br>
- Dra Sara Lumbreras (https://www.iit.comillas.edu/personas/slumbreras);
- Dr Javier García González (https://www.iit.comillas.edu/personas/javiergg); and 
- Dr Phillipe Vilaça Gomes (https://www.iit.comillas.edu/personas/phillipe.v).

**Prerequisite Knowledge Notice:** <br>
This educational material on "Hands-on Linear Programming with Gurobi" is designed for students who possess at least an intermediate level of understanding in key areas of Linear Programming (LP). Before delving into this content, it is recommended that students are familiar with the following concepts:

- **LP Formulation:** Understanding the basic mathematical formulation of linear programming problems.
- **LP Feasible Region and Optimality:** Knowledge of what constitutes a feasible region in LP and how optimality is determined within this context.
- **LP Sensitivity Analysis:** Insight into how changes in the parameters of a linear program affect its solution.
- **LP Duality:** A grasp of the principles of duality in linear programming, including the ability to understand and interpret dual problems.

This foundational knowledge is crucial for a comprehensive understanding of the material presented, as it builds upon these core concepts, particularly in the application of the Gurobi solver for LP problems.

## Table of Content
**[1. Presolve](#M1)** <br>
**[2. Eliminate Zero Rows​](#M2)** <br>
&nbsp;&nbsp;**[2.1 Theorem](#M2.1)** <br>
&nbsp;&nbsp;**[2.2 Modelling](#M2.2)** <br>
&nbsp;&nbsp;**[2.3 Implementation](#M2.3)** <br>
**[3. Eliminate Zero Rows​](#M3)** <br>
&nbsp;&nbsp;**[3.1 Theorem](#M3.1)** <br>
&nbsp;&nbsp;**[3.2 Modelling](#M3.2)** <br>
&nbsp;&nbsp;**[3.3 Implementation](#M3.3)** <br>
**[4. Eliminate Singleton Equality Constraints​](#M3)** <br>
&nbsp;&nbsp;**[4.1 Theorem](#M4.1)** <br>
&nbsp;&nbsp;**[4.2 Modelling](#M4.2)** <br>
&nbsp;&nbsp;**[4.3 Implementation](#M4.3)** <br>
**[4. Eliminate kton Equality Constraints](#M4)** <br>
**[5. Eliminate Singleton Inequality Constraints](#M5)** <br>
**[6. Eliminate Dual Singleton Inequality Constraints ](#M6)** <br>
**[7. Eliminate Implied Free Singleton Columns](#M7)** <br>
**[8. Eliminate Redundant Columns​](#M8)** <br>
**[9. Eliminate Implied Bounds on Rows ​](#M9)** <br>
**[10. Eliminate Redundant Rows​](#M10)** <br>
**[11. Make Coefficient Matrix Structurally Full Rank](#M11)** <br>

<a id="M1"> </a>
<span style="font-family: Arial; font-weight:bold;font-size:1.8em;color:#6E1B1B">1. Presolve

"Presolve Reductions in Mixed Integer Programming", Achterberg, et al, 2016.
- Presolve is a set of routines that remove redundant information and strengthen a given model formulation with the aim of accelerating the subsequent solution process.
- Presolve can be very effective; indeed, in some cases it is the difference between a problem being intractable and solvable.

**Importing some libraries**

In [263]:
import gurobipy as gp
from gurobipy import GRB
import os
import numpy as np
import warnings

# Suppress DeprecationWarning
warnings.filterwarnings('ignore', category=DeprecationWarning)

**Initial Model**

In [264]:
# Define the model
model = gp.Model("FactoryOptimization")

# Add Decision Variables
x1 = model.addVar(vtype=GRB.CONTINUOUS, name="x1", lb=0)
x2 = model.addVar(vtype=GRB.CONTINUOUS, name="x2", lb=0)

# Set the Objective Function
model.setObjective(50 * x1 + 40 * x2, GRB.MAXIMIZE)

# Machine 1 time constraint
model.addConstr(x1 + 0.5 * x2 <= 12, "Machine1")

# Machine 2 time constraint
model.addConstr(0.5 * x1 + x2 <= 10, "Machine2")

# Market demand constraints
model.addConstr(x1 <= 20, "DemandA")
model.addConstr(x2 <= 25, "DemandB")

<gurobi.Constr *Awaiting Model Update*>

In [265]:
# print the model (build from mps file)
model.update()
model.display()

Maximize
  50.0 x1 + 40.0 x2
Subject To
  Machine1: x1 + 0.5 x2 <= 12
  Machine2: 0.5 x1 + x2 <= 10
  DemandA: x1 <= 20
  DemandB: x2 <= 25


In [266]:
# Solve the Model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4 rows, 2 columns and 6 nonzeros
Model fingerprint: 0xbeafecd7
Coefficient statistics:
  Matrix range     [5e-01, 1e+00]
  Objective range  [4e+01, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 3e+01]
Presolve removed 2 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.6000000e+02   1.399450e+01   0.000000e+00      0s
       2    6.8000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  6.800000000e+02


In [267]:
# Print the solution
if model.status == GRB.OPTIMAL:
    print(f"Optimal number of Product A to produce: {x1.X}")
    print(f"Optimal number of Product B to produce: {x2.X}")
    print(f"Maximum Profit: ${model.ObjVal}")

Optimal number of Product A to produce: 9.333333333333334
Optimal number of Product B to produce: 5.333333333333333
Maximum Profit: $680.0


<a id="M2"> </a>
<span style="font-family: Arial; font-weight:bold;font-size:1.8em;color:#6E1B1B">2. Eliminate Zero Rows​

<a id="M2.1"> </a>
<span style="font-family: Arial; font-weight:bold;font-size:1.0em;color:#6E1B1B">2.1 Theorem 

In linear programming, a row of the coefficient matrix A is considered an empty row if all the coefficients in that row are equal to zero. 
Such a zero row can be formulated as follows:

$ A_{i1}*x_1 + A_{i2}*x_2 + ... + A_{in}*x_n \; o_i \; b_i $

Where:
- $o_i$ represents the i-th operation, belonging to the set {<=, =, >=}.
- $A_{ij} = 0$ for all i = 1, 2, ..., m, and j = 1, 2, ..., n.

A constraint of this type may either be redundant or may indicate that the LP problem is infeasible. 
This distinction is made clear in the following theorem, considering all possible cases where $A_{i.}$ is an empty row:

- Case 1: if $o_i$ is "<=" and $b_i >= 0$, the constraint is redundant.
- Case 2: if $o_i$ is "<=" and $b_i < 0$, the constraint is infeasible.
- Case 3: if $o_i$ is " >=" and $b_i <= 0$, the constraint is redundant.
- Case 4: if $o_i$ is " >=" and $b_i > 0$, the constraint is infeasible.
- Case 5: if $o_i$ is " =" and $b_i = 0$, the constraint is redundant.
- Case 6: if $o_i$ is " =" and $b_i \neq 0$, the constraint is infeasible.

<a id="M2.2"> </a>
<span style="font-family: Arial; font-weight:bold;font-size:1.0em;color:#6E1B1B">2.2 Modelling 

In [255]:
# add these constraints to the problem
model.addConstr(0 * x1 + 0* x2 <= 10, "zero_rows_case_1")
model.addConstr(0 * x1 + 0* x2 <= -10, "zero_rows_case_2")
model.addConstr(0 * x1 + 0* x2 >= -10, "zero_rows_case_3")
model.addConstr(0 * x1 + 0* x2 >= 10, "zero_rows_case_4")
model.addConstr(0 * x1 + 0* x2 == 0, "zero_rows_case_5")
model.addConstr(0 * x1 + 0* x2 == 10, "zero_rows_case_6")

<gurobi.Constr *Awaiting Model Update*>

In [256]:
#updating the model
model.update()

In [257]:
# print the model (build from mps file)
model.display()

Maximize
  50.0 x1 + 40.0 x2
Subject To
  Machine1: x1 + 0.5 x2 <= 12
  Machine2: 0.5 x1 + x2 <= 10
  DemandA: x1 <= 20
  DemandB: x2 <= 25
  zero_rows_case_1: 0.0 <= 10
  zero_rows_case_2: 0.0 <= -10
  zero_rows_case_3: 0.0 >= -10
  zero_rows_case_4: 0.0 >= 10
  zero_rows_case_5: 0.0 = 0
  zero_rows_case_6: 0.0 = 10


In [258]:
# Access constraint matrix A
A = model.getA()
print(A.A)

[[1.  0.5]
 [0.5 1. ]
 [1.  0. ]
 [0.  1. ]
 [0.  0. ]
 [0.  0. ]
 [0.  0. ]
 [0.  0. ]
 [0.  0. ]
 [0.  0. ]]


<a id="M2.3"> </a>
<span style="font-family: Arial; font-weight:bold;font-size:1.0em;color:#6E1B1B">2.3 Implementation

In [261]:
def eliminate_zero_rows(model):
    """
    Eliminate zero rows from a Gurobi optimization model.

    This function first creates a copy of the given model. It then checks each 
    constraint to identify zero rows, i.e., rows where all coefficients are zero.

    Based on the right-hand side and the sense of these zero rows, the function 
    classifies each constraint as:
    - 'Redundant' if the constraint is rendered redundant based on the rules
    - 'Infeasible' if the constraint renders the model infeasible
    - 'Valid' for all other constraints

    Parameters:
    - model: The Gurobi model to be processed.

    Returns:
    - A dictionary where each key is a constraint name, and the value is
      a string indicating whether the constraint is 'Redundant', 'Infeasible', or 'Valid'.
    """
    try:
        # Copy the model
        model_copy = model.copy()

        # Identify zero rows and classify constraints
        feedback = {}
        for i, constr in enumerate(model_copy.getConstrs()):
            is_zero_row = all(model_copy.getCoeff(constr, var) == 0 for var in model_copy.getVars())

            if is_zero_row:
                if constr.Sense == GRB.LESS_EQUAL:
                    if constr.RHS >= 0:
                        feedback[constr.ConstrName] = 'Redundant'
                    else:
                        feedback[constr.ConstrName] = 'Infeasible'
                elif constr.Sense == GRB.GREATER_EQUAL:
                    if constr.RHS <= 0:
                        feedback[constr.ConstrName] = 'Redundant'
                    else:
                        feedback[constr.ConstrName] = 'Infeasible'
                elif constr.Sense == GRB.EQUAL:
                    if constr.RHS == 0:
                        feedback[constr.ConstrName] = 'Redundant'
                    else:
                        feedback[constr.ConstrName] = 'Infeasible'
            else:
                feedback[constr.ConstrName] = 'Valid'

        return feedback

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [262]:
feedback = eliminate_zero_rows(model)
for key, value in feedback.items():
    print(f'{key}: {value}')

Machine1: Valid
Machine2: Valid
DemandA: Valid
DemandB: Valid
zero_rows_case_1: Redundant
zero_rows_case_2: Infeasible
zero_rows_case_3: Redundant
zero_rows_case_4: Infeasible
zero_rows_case_5: Redundant
zero_rows_case_6: Infeasible


<a id="M3"> </a>
<span style="font-family: Arial; font-weight:bold;font-size:1.8em;color:#6E1B1B">3. Eliminate Zero Columns​

<a id="M3.1"> </a>
<span style="font-family: Arial; font-weight:bold;font-size:1.0em;color:#6E1B1B">3.1 Theorem 

In the context of linear programming, the process of eliminating zero columns involves analyzing the columns of the coefficient matrix, A. A column in this matrix is considered an empty column if all the coefficients in that column are equal to zero. In such scenarios, the corresponding variable might be either redundant or indicative of an unbounded linear programming (LP) problem. This distinction is made clear in the following theorem:

For each empty column in the coefficient matrix, A, we distinguish the following cases:

- **Case 1:** $c_j \geq 0$ <br>
  In this case, the variable is redundant and can be removed. This involves deleting the j-th variable from the decision vector, the j-th element in the cost vector, and the j-th column of the A matrix.

- **Case 2:** $c_j < 0$ <br>
   If this condition is met, it implies that the LP problem is unbounded.

<a id="M3.2"> </a>
<span style="font-family: Arial; font-weight:bold;font-size:1.0em;color:#6E1B1B">3.2 Modelling 

In [151]:
# Add a new Decision Variable
x3 = model.addVar(vtype=GRB.CONTINUOUS, name="x3", lb=0)
x4 = model.addVar(vtype=GRB.CONTINUOUS, name="x4", lb=0)
# Set the Objective Function
model.setObjective(50 * x1 + 40 * x2 - 10 * x3 + 8* x4, GRB.MAXIMIZE)

In [152]:
model.update()

In [153]:
# print the model (build from mps file)
model.display()

Maximize
  50.0 x1 + 40.0 x2 + -10.0 x3 + 8.0 x4
Subject To
  Machine1: x1 + 0.5 x2 <= 12
  Machine2: 0.5 x1 + x2 <= 10
  DemandA: x1 <= 20
  DemandB: x2 <= 25
  zero_rows_case_1: 0.0 <= 10
  zero_rows_case_2: 0.0 <= -10
  zero_rows_case_3: 0.0 >= -10
  zero_rows_case_4: 0.0 >= 10
  zero_rows_case_5: 0.0 = 0
  zero_rows_case_6: 0.0 = 10


In [154]:
# Access constraint matrix A
A = model.getA()
print(A.A)

[[1.  0.5 0.  0. ]
 [0.5 1.  0.  0. ]
 [1.  0.  0.  0. ]
 [0.  1.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]]


<a id="M3.3"> </a>
<span style="font-family: Arial; font-weight:bold;font-size:1.0em;color:#6E1B1B">3.3 Implementation

In [155]:
def eliminate_zero_columns(model):
    """
    This function evaluates each decision variable in a given Gurobi model to identify redundant or unbounded variables.
    It checks if a column in the coefficient matrix A is empty (all coefficients are zero).
    Depending on the cost coefficient c_j, the variable is classified as redundant, unbounded, or valid.

    Args:
    model (gurobipy.Model): The Gurobi model to evaluate.

    Returns:
    dict: A dictionary where keys are variable names and values are feedback strings ('Redundant', 'Unbounded', 'Valid').
    """

    feedback = {}
    A = model.getA()
    for j, var in enumerate(model.getVars()):
        # Extract the column corresponding to the variable
        col = A.A[:, j]

        # Check if all coefficients in the column are zero
        if all(c == 0 for c in col):
            c_j = var.obj

            # Classify based on c_j value
            if c_j >= 0:
                feedback[var.varName] = 'Redundant'
            else:
                feedback[var.varName] = 'Unbounded'
        else:
            feedback[var.varName] = 'Valid'

    return feedback

In [156]:
feedback_var = eliminate_zero_columns(model)
for key, value in feedback_var.items():
    print(f'{key}: {value}')

x1: Valid
x2: Valid
x3: Unbounded
x4: Redundant


<a id="M4"> </a>
<span style="font-family: Arial; font-weight:bold;font-size:1.8em;color:#6E1B1B">4. Eliminate Singleton Equality Constraints​

<a id="M4.1"> </a>
<span style="font-family: Arial; font-weight:bold;font-size:1.0em;color:#6E1B1B">4.1 Theorem 

An equality row of the coefficient matrix A is a singleton row if one coefficient in that row is nonzero. This can be described in the context of a linear program as follows:

The singleton equality row can be expressed as:

$ A_{i1} \cdot x_1 + A_{i2} \cdot x_2 + \ldots + A_{in} \cdot x_n = b_i $
- where $A_{ik} \neq 0 $ and $A_{ij} = 0 $, $i \in \{1, 2, ..., m\}$, $j \in \{1, 2, ..., n\}$, and for $j \neq k$. 
- The constraint simplifies to: $A_{ik} \cdot x_k = b_i$
- Hence, $x_k = \frac{b_i}{A_{ik}}$.
- Depending on this value, we can determine the nature of the constraint:

**Case 1: $x_k \geq 0$** <br>
- Row *i* and column *k* are redundant and can be deleted.
    - Adjustments:
        - Replace variable $x_k$ in all constraints: $b := b - x_k \cdot A_{.k}$
        - If $c_k \neq 0$, update the constant term of the objective function as $c_o = c_o - c_k \cdot \left( \frac{b_i}{A_{ik}} \right)$
    - Deletions:
        - Delete row $i$ from matrix $A$, delete element $i$ from vector b
        - Delete column *k* from matrix A, delete element *k* from vector c
    - Note: This process can create new singleton equality rows, so it's repeated until no more such rows exist.


**Case 2: $x_k < 0$**
- The linear programming problem is infeasible.

<a id="M4.2"> </a>
<span style="font-family: Arial; font-weight:bold;font-size:1.0em;color:#6E1B1B">4.2 Modelling 

In [157]:
model.update()
model.display()

Maximize
  50.0 x1 + 40.0 x2 + -10.0 x3 + 8.0 x4
Subject To
  Machine1: x1 + 0.5 x2 <= 12
  Machine2: 0.5 x1 + x2 <= 10
  DemandA: x1 <= 20
  DemandB: x2 <= 25
  zero_rows_case_1: 0.0 <= 10
  zero_rows_case_2: 0.0 <= -10
  zero_rows_case_3: 0.0 >= -10
  zero_rows_case_4: 0.0 >= 10
  zero_rows_case_5: 0.0 = 0
  zero_rows_case_6: 0.0 = 10


In [158]:
# add these constraints to the problem
model.addConstr(0 * x1 + 0* x2 + 0* x3 + 4 * x4 == 10, "single_equal_case_1")
model.addConstr(0 * x1 + 0* x2 + 0* x3 + 4 * x4 == -10, "single_equal_case_2")

<gurobi.Constr *Awaiting Model Update*>

In [159]:
model.update()
model.display()

Maximize
  50.0 x1 + 40.0 x2 + -10.0 x3 + 8.0 x4
Subject To
  Machine1: x1 + 0.5 x2 <= 12
  Machine2: 0.5 x1 + x2 <= 10
  DemandA: x1 <= 20
  DemandB: x2 <= 25
  zero_rows_case_1: 0.0 <= 10
  zero_rows_case_2: 0.0 <= -10
  zero_rows_case_3: 0.0 >= -10
  zero_rows_case_4: 0.0 >= 10
  zero_rows_case_5: 0.0 = 0
  zero_rows_case_6: 0.0 = 10
  single_equal_case_1: 4.0 x4 = 10
  single_equal_case_2: 4.0 x4 = -10


In [160]:
# Access constraint matrix A
A = model.getA()
print(A.A)

[[1.  0.5 0.  0. ]
 [0.5 1.  0.  0. ]
 [1.  0.  0.  0. ]
 [0.  1.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  4. ]
 [0.  0.  0.  4. ]]


<a id="M4.3"> </a>
<span style="font-family: Arial; font-weight:bold;font-size:1.0em;color:#6E1B1B">4.3 Implementation

In [244]:
def eliminate_singleton_equalities(model, current_matrices_path):
    """
    This function processes a Gurobi model to eliminate singleton equality constraints.
    It simplifies the model by fixing variables and updating the model accordingly.
    If a negative fixed value for a variable is found, the model is declared infeasible.

    Args:
    model (gurobipy.Model): The Gurobi model to process.

    Returns:
    gurobipy.Model: The updated Gurobi model after eliminating singleton equalities.
    """

    # Copy the model to avoid modifying the original
    copied_model = model.copy()

    # Variable to track if we found a singleton in the current iteration
    found_singleton = True

    # Dictionary to store solutions for singletons
    solution = {}

    while found_singleton:
        found_singleton = False

        # Getting the matrices of the model
        A, b, c, lb, ub, of_sense, cons_senses = get_model_matrices(copied_model)

        # Getting objective function expression
        of = copied_model.getObjective()

        # Getting the variable names
        variable_names = [var.VarName for var in copied_model.getVars()]

        # Getting the number of nonzero elements per row
        nonzero_count_per_row = np.count_nonzero(A.A, axis=1)

        # Create a boolean array where True indicates rows with a single non-zero element
        single_nonzero = nonzero_count_per_row == 1

        # Create a boolean array where True indicates rows with '=' constraint sense
        equality_constraints = np.array(cons_senses) == '='

        # Combine the two conditions
        valid_rows = np.logical_and(single_nonzero, equality_constraints)

        # Find the index of the first row that satisfies both conditions
        first_singleton_index = np.where(valid_rows)[0][0] if np.any(valid_rows) else None

        # Check if the equality singleton row exists and process it
        if first_singleton_index is not None:

            found_singleton = True
            # Extract the singleton row
            singleton_row = A.A[first_singleton_index, :]

            # Identify the non-zero column (k)
            k_index = np.nonzero(singleton_row)[0][0]

            # Calculate the value for the variable corresponding to the singleton row
            x_k = b[first_singleton_index] / A.A[first_singleton_index, k_index]

            if x_k >= 0:
                # Update the solution dictionary
                solution[variable_names[k_index]] = x_k

                # update b
                b = b - A.A[:, k_index] * x_k
                b = np.delete(b, first_singleton_index)
                b = b.tolist()

                # update objective function constant
                if c[k_index] != 0:
                    of = of - c[k_index]*x_k

                # Update A
                A_new = np.delete(A.A, first_singleton_index, axis=0)  # Delete row
                A_new = np.delete(A_new, k_index, axis=1)  # Delete column
                A = csr_matrix(A_new)

                # update c, lb, ub and cons_senses
                del c[k_index]
                lb = np.delete(lb, k_index)
                ub = np.delete(ub, k_index)
                del cons_senses[first_singleton_index]
                del variable_names[k_index]

                save_json(A, b, c, lb, ub, of_sense, cons_senses, current_matrices_path, variable_names)
                copied_model = build_model_from_json(current_matrices_path)

            else:
                # Problem is infeasible
                return "Warning: Model is infeasible due to a negative singleton."

    # Update the model
    copied_model.update()
    return copied_model, solution

In [245]:
model.update()
model.display()

Maximize
  -20.0 + 50.0 x1 + 40.0 x2 + -10.0 x3 + 8.0 x4
Subject To
  Machine1: x1 + 0.5 x2 <= 12
  Machine2: 0.5 x1 + x2 <= 10
  DemandA: x1 <= 20
  DemandB: x2 <= 25
  zero_rows_case_1: 0.0 <= 10
  zero_rows_case_2: 0.0 <= -10
  zero_rows_case_3: 0.0 >= -10
  zero_rows_case_4: 0.0 >= 10
  zero_rows_case_5: 0.0 = 0
  zero_rows_case_6: 0.0 = 10
  single_equal_case_1: 4.0 x4 = 10
  single_equal_case_2: 4.0 x4 = -10


In [247]:
new_model = eliminate_singleton_equalities(model)

GurobiError: Constr was removed from the model

<a id="M4"> </a>
<span style="font-family: Arial; font-weight:bold;font-size:1.8em;color:#6E1B1B">4. Eliminate kton Equality Constraints

<a id="M5"> </a>
<span style="font-family: Arial; font-weight:bold;font-size:1.8em;color:#6E1B1B">5. Eliminate Singleton Inequality Constraints

<a id="M6"> </a>
<span style="font-family: Arial; font-weight:bold;font-size:1.8em;color:#6E1B1B">6. Eliminate Dual Singleton Inequality Constraints

<a id="M7"> </a>
<span style="font-family: Arial; font-weight:bold;font-size:1.8em;color:#6E1B1B">7. Eliminate Implied Free Singleton Columns

<a id="M8"> </a>
<span style="font-family: Arial; font-weight:bold;font-size:1.8em;color:#6E1B1B">8. Eliminate Redundant Columns​

<a id="M9"> </a>
<span style="font-family: Arial; font-weight:bold;font-size:1.8em;color:#6E1B1B">9. Eliminate Implied Bounds on Rows ​

<a id="M10"> </a>
<span style="font-family: Arial; font-weight:bold;font-size:1.8em;color:#6E1B1B">10. Eliminate Redundant Rows​

<a id="M11"> </a>
<span style="font-family: Arial; font-weight:bold;font-size:1.8em;color:#6E1B1B">11. Make Coefficient Matrix Structurally Full Rank